In [5]:

"""
Módulo de funções de ativação para redes neurais.

Cada função de ativação tem:
- forward: calcula a ativação
- derivada: usada no backpropagation (veremos depois)
"""

class Ativacao:
    """"Classe base para funções de ativação. """

    def forward(self, x):
        """Aplica a função de ativação """
        raise NotImplementedError

    def derivada(self, x):
        """Calcula a derivada (para backpropagation) """
        raise NotImplementedError


In [6]:
# === Funções de ativação

class Degrau(Ativacao):
    """
    Função Degrau (Step Function).
    Retorna 1 se x >= limiar, senão 0.
    """

    def __init__(self, limiar=1.0):
        self.limiar = limiar

    def forward(self, x):
        return 1 if x >= self.limiar else 0

    def derivada(self, x):
        # Degrau não é diferenciável, mas podemos retornar 0
        return 0

class Sigmoid(Ativacao):
    """
    Função Sigmoid (Logística).
    Mapeia valores para range (0, 1).
    """

    def forward(self, x):
        return 1 / (1 + np.exp(-x))

    def derivada(self, x):
        s = self.forward(x)
        return s * (1 - s)

class ReLU(Ativacao):
    """
    Rectified Linear Unit.
    Retorna max(0, x).
    """

    def forward(self, x):
        return max(0, x)

    def derivada(self, x):
        return 1 if 0 >= 0 else 0

class Tanh(Ativacao):
    """
    Tangente Hiperbólica.
    Mapeia valores para range (-1, 1).
    """

    def forward(self, x):
        return np.tanh(x)

    def derivada(self, x):
        return 1 - np.tanh(x)**2

class Linear(Ativacao):

    def forward(self, x):
        return x

    def derivada(self, x):
        s = self.foward(x)
        return s
    
    

In [10]:
"""
Implementação de um neurônio artificial
"""

# from activations import Ativacao, Degrau

class Neuronio:
    """
    Um neurônio artificial com aprendizado supervisionado.
    
    Attributes:
        pesos (np.ndarray): Pesos sinápticos
        bias (float): Termo de viés
        taxa_aprendizado (float): Taxa de aprendizado
        ativacao (Ativacao): Função de ativação
    """
    
    def __init__(self, num_entradas, taxa_aprendizado=0.1, ativacao=None):
    
        """
        Inicializa o neurônio.
        
        Args:
            num_entradas (int): Número de entradas
            taxa_aprendizado (float): Taxa de aprendizado
            ativacao (Ativacao): Função de ativação (padrão: Degrau)
        """
        self.pesos = np.random.rand(num_entradas)
        self.bias = 0.0
        self.taxa_aprendizado = taxa_aprendizado
        
        # Se não passar ativação, usa Degrau por padrão
        self.ativacao = ativacao if ativacao is not None else Degrau()


    def forward(self, entrada):
        """
        Propagação para frente.
        
        Args:
            entrada: Vetor de entrada
            
        Returns:
            tuple: (saida_bruta, saida_ativada)
        """

        # Combinação linear
        saida_bruta = np.dot(entrada, self.pesos) + self.bias

        # Aplica ativação (agora vem do objeto de ativação)
        saida_ativada = self.ativacao.forward(saida_bruta)

        return saida_bruta, saida_ativada

    def treinar(self, entrada, esperado):
        """
        Treina o neurônio com um exemplo.
        
        Args:
            entrada: Vetor de entrada
            esperado: Saída esperada
            
        Returns:
            tuple: (saida, erro)
        """
        saida_bruta, saida = self.forward(entrada)
        erro = esperado - saida

        # Atualiza pesos
        self.pesos += self.taxa_aprendizado * erro * np.array(entrada)
        self.bias += self.taxa_aprendizado * erro

        return saida, erro

    def __repr__(self):
        """Representação em string do neurônio."""
        return f"Neuronio(entradas={len(self.pesos)}, ativacao={self.ativacao.__class__.__name__})"

        
    

In [11]:
"""
Implementação de uma camada de neurônios.
"""

import numpy as np

class Camada:
    """
    Uma camada (layer) de neurônios.
    
    Todos os neurônios da camada:
    - Recebem a mesma entrada
    - Podem ter ativações diferentes (ou mesma)
    - Produzem um vetor de saídas
    
    Attributes:
        neuronios (list): Lista de objetos Neuronio
        num_neuronios (int): Quantidade de neurônios na camada
        num_entradas (int): Número de entradas que cada neurônio recebe
    """
    
    def __init__(self, num_entradas, num_neuronios, taxa_aprendizado=0.1, ativacao=None):
        """
        Inicializa a camada.
        
        Args:
            num_entradas (int): Número de entradas para cada neurônio
            num_neuronios (int): Quantidade de neurônios na camada
            taxa_aprendizado (float): Taxa de aprendizado
            ativacao (Ativacao): Função de ativação (padrão: Degrau)
        """
        self.num_neuronios = num_neuronios
        self.num_entradas = num_entradas
        
        # Cria uma lista de neurônios
        self.neuronios = []
        for _ in range(num_neuronios):
            neuronio = Neuronio(
                num_entradas=num_entradas,
                taxa_aprendizado=taxa_aprendizado,
                ativacao=ativacao if ativacao is not None else Degrau()
            )
            self.neuronios.append(neuronio)
    
    def forward(self, entrada):
        """
        Propagação para frente através da camada.
        
        Args:
            entrada: Vetor de entrada (mesmo para todos os neurônios)
            
        Returns:
            tuple: (saidas_brutas, saidas_ativadas)
                - saidas_brutas: lista com saídas antes da ativação
                - saidas_ativadas: lista com saídas após ativação
        """
        saidas_brutas = []
        saidas_ativadas = []
        
        # Cada neurônio processa a mesma entrada
        for neuronio in self.neuronios:
            bruta, ativada = neuronio.forward(entrada)
            saidas_brutas.append(bruta)
            saidas_ativadas.append(ativada)
        
        return np.array(saidas_brutas), np.array(saidas_ativadas)
    
    def treinar(self, entrada, esperados):
        """
        Treina todos os neurônios da camada.
        
        Args:
            entrada: Vetor de entrada
            esperados: Lista de saídas esperadas (uma para cada neurônio)
            
        Returns:
            tuple: (saidas, erros)
        """
        if len(esperados) != self.num_neuronios:
            raise ValueError(
                f"esperados deve ter {self.num_neuronios} valores, "
                f"mas recebeu {len(esperados)}"
            )
        
        saidas = []
        erros = []
        
        # Treina cada neurônio com sua saída esperada
        for neuronio, esperado in zip(self.neuronios, esperados):
            saida, erro = neuronio.treinar(entrada, esperado)
            saidas.append(saida)
            erros.append(erro)
        
        return np.array(saidas), np.array(erros)
    
    def get_pesos(self):
        """
        Retorna os pesos de todos os neurônios como matriz.
        
        Returns:
            np.ndarray: Matriz de pesos (num_neuronios x num_entradas)
        """
        pesos_matriz = []
        for neuronio in self.neuronios:
            pesos_matriz.append(neuronio.pesos)
        return np.array(pesos_matriz)
    
    def get_bias(self):
        """
        Retorna os bias de todos os neurônios.
        
        Returns:
            np.ndarray: Vetor de bias
        """
        return np.array([neuronio.bias for neuronio in self.neuronios])
    
    def __repr__(self):
        """Representação em string da camada."""
        ativacao_nome = self.neuronios[0].ativacao.__class__.__name__
        return (
            f"Camada(neuronios={self.num_neuronios}, "
            f"entradas={self.num_entradas}, "
            f"ativacao={ativacao_nome})"
        )
    
    def __len__(self):
        """Retorna o número de neurônios."""
        return self.num_neuronios


class CamadaVetorizada:
    """
    Versão OTIMIZADA da camada usando operações vetorizadas.
    
    Muito mais rápida que criar neurônios individuais!
    Essa é a abordagem usada em bibliotecas profissionais.
    """
    
    def __init__(self, num_entradas, num_neuronios, taxa_aprendizado=0.1, ativacao=None):
        """
        Inicializa a camada vetorizada.
        
        Args:
            num_entradas (int): Dimensão da entrada
            num_neuronios (int): Número de neurônios (dimensão da saída)
            taxa_aprendizado (float): Taxa de aprendizado
            ativacao (Ativacao): Função de ativação
        """
        self.num_neuronios = num_neuronios
        self.num_entradas = num_entradas
        self.taxa_aprendizado = taxa_aprendizado
        self.ativacao = ativacao if ativacao is not None else Degrau()
        
        # Matriz de pesos: (num_neuronios x num_entradas)
        # Cada LINHA é um neurônio
        self.pesos = np.random.rand(num_neuronios, num_entradas)
        
        # Vetor de bias: (num_neuronios,)
        self.bias = np.zeros(num_neuronios)
    
    def forward(self, entrada):
        """
        Propagação vetorizada (MUITO mais rápida!).
        
        Args:
            entrada: Vetor de entrada (num_entradas,)
            
        Returns:
            tuple: (saidas_brutas, saidas_ativadas)
        """
        # Multiplicação matricial: (num_neuronios x num_entradas) @ (num_entradas,)
        # Resultado: (num_neuronios,)
        saidas_brutas = self.pesos @ entrada + self.bias
        
        # Aplica ativação em cada elemento
        saidas_ativadas = np.array([
            self.ativacao.forward(x) for x in saidas_brutas
        ])
        
        return saidas_brutas, saidas_ativadas
    
    def treinar(self, entrada, esperados):
        """
        Treinamento vetorizado.
        
        Args:
            entrada: Vetor de entrada
            esperados: Vetor de saídas esperadas
            
        Returns:
            tuple: (saidas, erros)
        """
        saidas_brutas, saidas = self.forward(entrada)
        erros = esperados - saidas
        
        # Atualização vetorizada dos pesos
        # Cada linha (neurônio) é atualizada: peso += taxa * erro * entrada
        entrada_array = np.array(entrada)
        for i in range(self.num_neuronios):
            self.pesos[i] += self.taxa_aprendizado * erros[i] * entrada_array
            self.bias[i] += self.taxa_aprendizado * erros[i]
        
        return saidas, erros
    
    def __repr__(self):
        return (
            f"CamadaVetorizada(neuronios={self.num_neuronios}, "
            f"entradas={self.num_entradas}, "
            f"ativacao={self.ativacao.__class__.__name__})"
        )

In [9]:
"""
Testes para a classe Camada.
"""

import numpy as np


print("=" * 60)
print("TESTE 1: Camada com 3 neurônios")
print("=" * 60)

# Cria uma camada: 10 entradas → 3 neurônios
camada = Camada(
    num_entradas=10,
    num_neuronios=3,
    taxa_aprendizado=0.1,
    ativacao=Sigmoid()
)

print(f"\n{camada}")
print(f"Número de neurônios: {len(camada)}")

# Entrada de teste
entrada = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

# Saídas esperadas para cada neurônio
esperados = [1, 0, 1]  # Neurônio 1: quer 1, Neurônio 2: quer 0, Neurônio 3: quer 1

print("\n--- Treinamento ---")
for epoca in range(10):
    saidas, erros = camada.treinar(entrada, esperados)
    print(f"Época {epoca} | saídas={saidas} | erros={erros}")

# Testa após treinamento
print("\n--- Teste Final ---")
_, saidas_finais = camada.forward(entrada)
print(f"Entrada: {entrada}")
print(f"Saídas: {saidas_finais}")
print(f"Esperado: {esperados}")


print("\n" + "=" * 60)
print("TESTE 2: Comparando Camada vs CamadaVetorizada")
print("=" * 60)

# Mesma configuração para ambas
entrada = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
esperados = np.array([1, 0, 1])

# Camada normal
camada_normal = Camada(10, 3, taxa_aprendizado=0.1, ativacao=Sigmoid())

# Camada vetorizada
camada_vet = CamadaVetorizada(10, 3, taxa_aprendizado=0.1, ativacao=Sigmoid())

print("\nTreinando ambas por 100 épocas...")

# Treina camada normal
for _ in range(100):
    camada_normal.treinar(entrada, esperados)

# Treina camada vetorizada
for _ in range(100):
    camada_vet.treinar(entrada, esperados)

# Compara resultados
_, saidas_normal = camada_normal.forward(entrada)
_, saidas_vet = camada_vet.forward(entrada)

print(f"\nCamada Normal: {saidas_normal}")
print(f"Camada Vetorizada: {saidas_vet}")
print(f"Esperado: {esperados}")


print("\n" + "=" * 60)
print("TESTE 3: Visualizando Pesos da Camada")
print("=" * 60)

camada = Camada(5, 2, ativacao=ReLU())

print(f"\nPesos iniciais:")
print(camada.get_pesos())
print(f"\nBias iniciais:")
print(camada.get_bias())

# Treina um pouco
entrada = [1, 0, 1, 0, 1]
esperados = [1, 0]

for _ in range(50):
    camada.treinar(entrada, esperados)

print(f"\nPesos após treinamento:")
print(camada.get_pesos())
print(f"\nBias após treinamento:")
print(camada.get_bias())

TESTE 1: Camada com 3 neurônios


<class 'NameError'>: name 'Neuronio' is not defined